In [1]:
from pulp import *
import pandas as pd

In [2]:
cost = pd.read_excel("transportation.xlsx", sheet_name = "Cost")

In [3]:
cost

,Unnamed: 0,Warehouse1,Warehouse2,Warehouse3
0,Supplier1,15,20,10
1,Supplier2,7,12,9
2,Supplier3,40,5,20


In [24]:
cost.dtypes

Unnamed: 0    object
Warehouse1     int64
Warehouse2     int64
Warehouse3     int64
dtype: object

In [4]:
parameter = pd.read_excel("transportation.xlsx", sheet_name = "Parameter")
parameter

,Capacity,Demand
0,1000,500
1,500,1000
2,2000,2000


In [8]:
cost.columns

Index(['Unnamed: 0', 'Warehouse1', 'Warehouse2', 'Warehouse3'], dtype='object')

In [7]:
#Step1:Set and Parameters
warehouse = [i for i in cost.columns if i != cost.columns[0]]
print(warehouse)

['Warehouse1', 'Warehouse2', 'Warehouse3']


In [9]:
cost.iloc[:,0] #: means selecting each row
#iloc = ilocation, :=เอาข้อมูลทุกแถว, 0=เอาเฉพาะcolumnแรก = column [0]

0    Supplier1
1    Supplier2
2    Supplier3
Name: Unnamed: 0, dtype: object

In [10]:
supplier = cost.iloc[:,0].values # .values แปลง Dataframe ให้เป็นlist

In [11]:
supplier

array(['Supplier1', 'Supplier2', 'Supplier3'], dtype=object)

In [16]:
cost

,Unnamed: 0,Warehouse1,Warehouse2,Warehouse3
0,Supplier1,15,20,10
1,Supplier2,7,12,9
2,Supplier3,40,5,20


In [17]:
cost_dict={}
for i in range(len(supplier)): #0,1,2
    for j in range(len(warehouse)): #0,1,2
       cost_dict[supplier[i],warehouse[j]] = cost.iloc[i,j+1] #ใช้ ilocต้องเป็นตัวเลข

In [18]:
print(cost_dict)

{('Supplier1', 'Warehouse1'): 15, ('Supplier1', 'Warehouse2'): 20, ('Supplier1', 'Warehouse3'): 10, ('Supplier2', 'Warehouse1'): 7, ('Supplier2', 'Warehouse2'): 12, ('Supplier2', 'Warehouse3'): 9, ('Supplier3', 'Warehouse1'): 40, ('Supplier3', 'Warehouse2'): 5, ('Supplier3', 'Warehouse3'): 20}


In [19]:
parameter.iloc[:,0]

0    1000
1     500
2    2000
Name: Capacity, dtype: int64

In [20]:
capacity_dict = {}
for i in range(len(supplier)): 
  capacity_dict[supplier[i]] = parameter.iloc[:,0][i]
print(capacity_dict) 
 

{'Supplier1': 1000, 'Supplier2': 500, 'Supplier3': 2000}


In [22]:
parameter

,Capacity,Demand
0,1000,500
1,500,1000
2,2000,2000


In [21]:
parameter.iloc[:,1]

0     500
1    1000
2    2000
Name: Demand, dtype: int64

In [28]:
demand_dict = {}
for i in range(len(warehouse)): 
  demand_dict[warehouse[i]] = parameter.iloc[:,1][i]
print(demand_dict) 

{'Warehouse1': 500, 'Warehouse2': 1000, 'Warehouse3': 2000}


In [29]:
cost_dict

{('Supplier1', 'Warehouse1'): 15,
 ('Supplier1', 'Warehouse2'): 20,
 ('Supplier1', 'Warehouse3'): 10,
 ('Supplier2', 'Warehouse1'): 7,
 ('Supplier2', 'Warehouse2'): 12,
 ('Supplier2', 'Warehouse3'): 9,
 ('Supplier3', 'Warehouse1'): 40,
 ('Supplier3', 'Warehouse2'): 5,
 ('Supplier3', 'Warehouse3'): 20}

## Linear Programming Model

In [38]:
#Step:Define problem
#Linear Programming Model
prob = LpProblem("Trasportation Problem",LpMinimize) #ใช้คำสั่ง LpProblem สร้าง Model prob,บอกปัญหาใช้ LpMaximize หรือ LpMinimize

#Step:Decision variable
x=LpVariable.dicts("x",cost_dict.keys(),0)#ใช้ cost_dict.keys()เพื่อประกาศ index ให้ตัวแปร x



C:\Python Anaconda\lib\site-packages\pulp\pulp.py:1114: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [39]:
print(x)

{('Supplier1', 'Warehouse1'): x_('Supplier1',_'Warehouse1'), ('Supplier1', 'Warehouse2'): x_('Supplier1',_'Warehouse2'), ('Supplier1', 'Warehouse3'): x_('Supplier1',_'Warehouse3'), ('Supplier2', 'Warehouse1'): x_('Supplier2',_'Warehouse1'), ('Supplier2', 'Warehouse2'): x_('Supplier2',_'Warehouse2'), ('Supplier2', 'Warehouse3'): x_('Supplier2',_'Warehouse3'), ('Supplier3', 'Warehouse1'): x_('Supplier3',_'Warehouse1'), ('Supplier3', 'Warehouse2'): x_('Supplier3',_'Warehouse2'), ('Supplier3', 'Warehouse3'): x_('Supplier3',_'Warehouse3')}


In [40]:
cost_dict.keys()

dict_keys([('Supplier1', 'Warehouse1'), ('Supplier1', 'Warehouse2'), ('Supplier1', 'Warehouse3'), ('Supplier2', 'Warehouse1'), ('Supplier2', 'Warehouse2'), ('Supplier2', 'Warehouse3'), ('Supplier3', 'Warehouse1'), ('Supplier3', 'Warehouse2'), ('Supplier3', 'Warehouse3')])

In [41]:
cost_dict.values()

dict_values([15, 20, 10, 7, 12, 9, 40, 5, 20])

In [42]:
#step: Objective function
prob += lpSum(cost_dict[i]*x[i] for i in cost_dict.keys()) #cost_dict.keys มี index = 2 มิติแล้ว


In [43]:
prob 
# แปลผล เช่น costขนส่ง supplier1 ไป warehouse1 =15บาทต่อตันคูณกับx(จำนวนตัน)
#cost จะคูณทุกroute รวมกันออกมาจะได้เป็น total costขนส่ง

Trasportation_Problem:
MINIMIZE
15*x_('Supplier1',_'Warehouse1') + 20*x_('Supplier1',_'Warehouse2') + 10*x_('Supplier1',_'Warehouse3') + 7*x_('Supplier2',_'Warehouse1') + 12*x_('Supplier2',_'Warehouse2') + 9*x_('Supplier2',_'Warehouse3') + 40*x_('Supplier3',_'Warehouse1') + 5*x_('Supplier3',_'Warehouse2') + 20*x_('Supplier3',_'Warehouse3') + 0
VARIABLES
x_('Supplier1',_'Warehouse1') Continuous
x_('Supplier1',_'Warehouse2') Continuous
x_('Supplier1',_'Warehouse3') Continuous
x_('Supplier2',_'Warehouse1') Continuous
x_('Supplier2',_'Warehouse2') Continuous
x_('Supplier2',_'Warehouse3') Continuous
x_('Supplier3',_'Warehouse1') Continuous
x_('Supplier3',_'Warehouse2') Continuous
x_('Supplier3',_'Warehouse3') Continuous

In [45]:
#step:Adding constraints (Supplier)
#Supplier ต้องไม่supply ของเกิน cap ที่มี (จำนวนตัน)
#For example: consider
#supplier1 >>x11+x12+x13 <=1000
#supplier2 >>x21+x22+x23 <=500
for i in supplier:
  prob += lpSum(x[i,j] for j in warehouse) <=capacity_dict[i]


In [46]:
prob

Trasportation_Problem:
MINIMIZE
15*x_('Supplier1',_'Warehouse1') + 20*x_('Supplier1',_'Warehouse2') + 10*x_('Supplier1',_'Warehouse3') + 7*x_('Supplier2',_'Warehouse1') + 12*x_('Supplier2',_'Warehouse2') + 9*x_('Supplier2',_'Warehouse3') + 40*x_('Supplier3',_'Warehouse1') + 5*x_('Supplier3',_'Warehouse2') + 20*x_('Supplier3',_'Warehouse3') + 0
SUBJECT TO
_C1: x_('Supplier1',_'Warehouse1') + x_('Supplier1',_'Warehouse2')
 + x_('Supplier1',_'Warehouse3') <= 1000

_C2: x_('Supplier2',_'Warehouse1') + x_('Supplier2',_'Warehouse2')
 + x_('Supplier2',_'Warehouse3') <= 500

_C3: x_('Supplier3',_'Warehouse1') + x_('Supplier3',_'Warehouse2')
 + x_('Supplier3',_'Warehouse3') <= 2000

VARIABLES
x_('Supplier1',_'Warehouse1') Continuous
x_('Supplier1',_'Warehouse2') Continuous
x_('Supplier1',_'Warehouse3') Continuous
x_('Supplier2',_'Warehouse1') Continuous
x_('Supplier2',_'Warehouse2') Continuous
x_('Supplier2',_'Warehouse3') Continuous
x_('Supplier3',_'Warehouse1') Continuous
x_('Supplier3',_'War

In [47]:
#step:Adding constraints (Warehouse)
#Demand ต้องมากกว่าหรือเท่ากับของที่ต้องการ และต้องไม่ทำให้ของ Out stock
#For example: consider
#warehouse1 >>x11+x21+x31 <=1000

for j in warehouse:
  prob += lpSum(x[i,j] for i in supplier) >= demand_dict[j]

In [48]:
prob

Trasportation_Problem:
MINIMIZE
15*x_('Supplier1',_'Warehouse1') + 20*x_('Supplier1',_'Warehouse2') + 10*x_('Supplier1',_'Warehouse3') + 7*x_('Supplier2',_'Warehouse1') + 12*x_('Supplier2',_'Warehouse2') + 9*x_('Supplier2',_'Warehouse3') + 40*x_('Supplier3',_'Warehouse1') + 5*x_('Supplier3',_'Warehouse2') + 20*x_('Supplier3',_'Warehouse3') + 0
SUBJECT TO
_C1: x_('Supplier1',_'Warehouse1') + x_('Supplier1',_'Warehouse2')
 + x_('Supplier1',_'Warehouse3') <= 1000

_C2: x_('Supplier2',_'Warehouse1') + x_('Supplier2',_'Warehouse2')
 + x_('Supplier2',_'Warehouse3') <= 500

_C3: x_('Supplier3',_'Warehouse1') + x_('Supplier3',_'Warehouse2')
 + x_('Supplier3',_'Warehouse3') <= 2000

_C4: x_('Supplier1',_'Warehouse1') + x_('Supplier2',_'Warehouse1')
 + x_('Supplier3',_'Warehouse1') >= 500

_C5: x_('Supplier1',_'Warehouse2') + x_('Supplier2',_'Warehouse2')
 + x_('Supplier3',_'Warehouse2') >= 1000

_C6: x_('Supplier1',_'Warehouse3') + x_('Supplier2',_'Warehouse3')
 + x_('Supplier3',_'Warehouse3') 

In [49]:
#step :Solve problem
prob.solve() 
#แปลผล ถ้าได้ -1 หมายถึง modelแก้ปัญหาไม่ออก (invisible)

1

In [51]:
#Step : Show objective function
print(value(prob.objective)) 
#แปลผลคือ costขนส่ง ต่ำสุด = 38,500 บาท

38500.0


In [52]:
#Step : The optimal solution คำตอบของปัญหา
for i in prob.variables():
    print(i.name,"=",i.varValue)
#แปลผล แนะนำsolution ให้แต่ล่ะwarehouseให้สั่งของ supplier แต่ละเจ้าในจำนวนเท่านี้ที่ทำให้costขนส่งต่ำสุด
#สื่งที่ต้องระวัง supplier ต้องมีของมากกว่า Demand ถ้าน้อยกว่า Demand Modelจะrunไม่ออกต้องใส่ Dummy supplier เข้าไปเพื่อหลอก modelให้ run ผลออกมา

x_('Supplier1',_'Warehouse1') = 0.0
x_('Supplier1',_'Warehouse2') = 0.0
x_('Supplier1',_'Warehouse3') = 1000.0
x_('Supplier2',_'Warehouse1') = 500.0
x_('Supplier2',_'Warehouse2') = 0.0
x_('Supplier2',_'Warehouse3') = 0.0
x_('Supplier3',_'Warehouse1') = 0.0
x_('Supplier3',_'Warehouse2') = 1000.0
x_('Supplier3',_'Warehouse3') = 1000.0
